In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import time
import copy
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
DATA_PATH = "/content/drive/MyDrive/Dry_Bean/data/Dry_Bean_Dataset.xlsx"
dry_beans = pd.read_excel(DATA_PATH)
dry_beans.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


In [ ]:
class DryBeanDataset(Dataset):
    def __init__(self, path):
        data = pd.read_excel(path)
        self.X = torch.Tensor(np.array(data.iloc[:, :-1], dtype=np.float32))
        self.label_encoder = LabelEncoder()
        self.y = torch.Tensor(self.label_encoder.fit_transform(data.iloc[:, -1]))
        self.y_label_classes = self.label_encoder.classes_
        self.n_samples = data.shape[0]
    
    def __getitem__(self, item):
        return self.X[item], self.y[item]

    def __len__(self):
        return self.n_samples
    
    def inverse_encoder(self):
        return self.y_label_classes

In [85]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(16,200)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(200, 7)

    def forward(self, x):
        output = self.fc1(x)
        output = self.relu1(output)
        output = self.fc2(output)
        return output

In [ ]:
dry_bean = DryBeanDataset(DATA_PATH)

In [ ]:
X, y = dry_bean[:]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
len(X_train)

10888

In [ ]:
len(X_test)

2723

In [ ]:
train_data = TensorDataset(X_train, y_train.type(torch.LongTensor))
test_data = TensorDataset(X_test, y_test.type(torch.LongTensor))

In [120]:
batch_size = 150
learning_rate = 0.001
num_epoch = 300

In [121]:
network = MLP()
criterion = nn.CrossEntropyLoss()
optimizer_ft = torch.optim.Adam(network.parameters(), lr=learning_rate)

In [122]:
data = {"train": train_data, "val": test_data}
dataloaders = {x: torch.utils.data.DataLoader(data[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val']}
dataset_sizes = {x: len(data[x]) for x in ['train', 'val']}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [123]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [124]:
model_ft = train_model(network, criterion, optimizer_ft, num_epochs=num_epoch)

Epoch 0/299
----------
train Loss: 889.0454 Acc: 0.1371
val Loss: 606.4623 Acc: 0.1913

Epoch 1/299
----------
train Loss: 438.8286 Acc: 0.1593
val Loss: 380.1557 Acc: 0.2560

Epoch 2/299
----------
train Loss: 295.7517 Acc: 0.1616
val Loss: 366.0456 Acc: 0.1550

Epoch 3/299
----------
train Loss: 448.3113 Acc: 0.1587
val Loss: 594.2124 Acc: 0.3551

Epoch 4/299
----------
train Loss: 340.6316 Acc: 0.1712
val Loss: 355.8862 Acc: 0.2578

Epoch 5/299
----------
train Loss: 325.1796 Acc: 0.1744
val Loss: 464.3339 Acc: 0.1913

Epoch 6/299
----------
train Loss: 274.1700 Acc: 0.1830
val Loss: 281.8419 Acc: 0.0404

Epoch 7/299
----------
train Loss: 281.3843 Acc: 0.1706
val Loss: 216.0575 Acc: 0.3739

Epoch 8/299
----------
train Loss: 274.6108 Acc: 0.2139
val Loss: 325.9490 Acc: 0.1194

Epoch 9/299
----------
train Loss: 213.8609 Acc: 0.2177
val Loss: 227.7886 Acc: 0.2927

Epoch 10/299
----------
train Loss: 218.1306 Acc: 0.2419
val Loss: 100.9296 Acc: 0.3272

Epoch 11/299
----------
train L

device(type='cpu')